In [1]:
import sys
import numpy as np
from ipyfilechooser import FileChooser
from matplotlib import pyplot as plt

sys.path.append('../../')
from Pulses import STA_pulses
from QEnvs.QEnvWave import QEnvWave

fc = FileChooser('../../logs/')
display(fc)

2023-06-09 20:58:48.390285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 20:58:49.031320: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


FileChooser(path='/home/pabolojo/Escritorio/Curso_5/TFG/MLQTransfer/Codes/logs', filename='', title='', show_h…

In [41]:
episode = np.load(fc.selected, allow_pickle=True)

times = episode[1][:-1]
pulses = episode[2]
populations = episode[3][:, :-1]

In [42]:
num_qubits = pulses.shape[0] + 1
time_max = times[-1]
num_steps = len(times)
omega_max = np.max(pulses)
omega_s = 0.
if num_qubits > 3:
    omega_s = np.max(pulses[1:-1])

In [43]:
pulses_sta = STA_pulses(num_qubits, time_max, num_steps, omega_max)

In [44]:
def dummy_cost_function(env):
    return 0.0

In [45]:
env = QEnvWave(num_qubits=num_qubits,
                   time_max=time_max,
                   num_steps=num_steps - 1,
                   cost_function=dummy_cost_function,
                   omega_max=omega_max)

In [46]:
def calculate_populations(env, pulse):
    times = env.times
    populations = []
    for state in env.run_qevolution(pulse):
        populations.append(np.abs(state))
    return times, np.array(populations).reshape((num_steps, num_qubits))

In [47]:
times_sta, populations_sta = calculate_populations(env, pulses_sta)

In [16]:
# Save results
file_name = fc.selected.split('/')[-1].split('.')[0]
pulses = pulses.T
save = np.array([times, pulses, pulses_sta, populations, populations_sta])
np.save("Data/sota_rl_comparison_{}.npy".format(file_name), save)

/tmp/ipykernel_33072/2500396333.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  save = np.array([times, pulses, pulses_sta, populations, populations_sta])
